In [1]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import imblearn
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler,OrdinalEncoder

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (classification_report,accuracy_score,precision_score,
                             confusion_matrix,ConfusionMatrixDisplay,roc_auc_score,roc_curve)

import warnings
warnings.simplefilter("ignore")

In [2]:
def wrangle(filepath):
    df=pd.read_csv(filepath)
   
    return df

In [3]:
df = wrangle(r'C:\Users\kelvin\OneDrive\Desktop\file1\credit_risk_dataset_training.csv')
df.head()

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,22,59000,RENT,123.0,PERSONAL,D,35000,16.02,1,0.59,Y,3
1,21,9600,OWN,5.0,EDUCATION,B,1000,11.14,0,0.10,N,2
2,23,65500,RENT,4.0,MEDICAL,C,35000,15.23,1,0.53,N,2
3,21,9900,OWN,2.0,VENTURE,A,2500,7.14,1,0.25,N,2
4,26,77100,RENT,8.0,EDUCATION,B,35000,12.42,1,0.45,N,3


In [4]:
orde =OrdinalEncoder()
trans_one =orde.fit_transform(df)
df=pd.DataFrame(trans_one,columns=df.columns)
df

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,2.0,1782.0,3.0,35.0,4.0,3.0,694.0,237.0,1.0,59.0,1.0,1.0
1,1.0,20.0,2.0,5.0,1.0,1.0,5.0,92.0,0.0,10.0,0.0,0.0
2,3.0,2021.0,3.0,4.0,3.0,2.0,694.0,215.0,1.0,53.0,0.0,0.0
3,1.0,23.0,2.0,2.0,5.0,0.0,55.0,15.0,1.0,25.0,0.0,0.0
4,6.0,2366.0,3.0,8.0,1.0,1.0,694.0,127.0,1.0,45.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
22845,32.0,1998.0,3.0,4.0,4.0,3.0,594.0,225.0,1.0,31.0,1.0,17.0
22846,37.0,1519.0,0.0,1.0,4.0,2.0,179.0,149.0,0.0,11.0,0.0,28.0
22847,34.0,2999.0,0.0,4.0,4.0,0.0,545.0,21.0,0.0,15.0,0.0,17.0
22848,36.0,3148.0,0.0,5.0,4.0,1.0,488.0,100.0,0.0,10.0,0.0,24.0


In [5]:
df["loan_status"]=df["loan_status"].astype("int")

In [6]:
for i in df.columns:
    l= df[i].isna().sum()
    if l>0:
        print(i,"has",df[i].isna().sum(),"null values")
        df[i].fillna(value =df[i].mean(),inplace=True)

person_emp_length has 613 null values
loan_int_rate has 2147 null values


In [7]:
y = df[["loan_status"]]
X=df[[i for i in df.columns if "loan_status" not in  i]]
y.shape


(22850, 1)

In [8]:
import statsmodels.api as sm
from scipy.stats import logistic
import statsmodels.formula.api as smf

In [9]:
df.columns

Index(['person_age', 'person_income', 'person_home_ownership',
       'person_emp_length', 'loan_intent', 'loan_grade', 'loan_amnt',
       'loan_int_rate', 'loan_status', 'loan_percent_income',
       'cb_person_default_on_file', 'cb_person_cred_hist_length'],
      dtype='object')

In [10]:
model= smf.logit("loan_status~ person_age+person_income+person_home_ownership + person_emp_length + loan_intent + loan_grade + loan_amnt +loan_int_rate + loan_percent_income +cb_person_default_on_file + cb_person_cred_hist_length"
          ,data=df).fit()

Optimization terminated successfully.
         Current function value: 0.369163
         Iterations 7


In [11]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:            loan_status   No. Observations:                22850
Model:                          Logit   Df Residuals:                    22838
Method:                           MLE   Df Model:                           11
Date:                Sat, 23 Sep 2023   Pseudo R-squ.:                  0.2999
Time:                        23:17:13   Log-Likelihood:                -8435.4
converged:                       True   LL-Null:                       -12049.
Covariance Type:            nonrobust   LLR p-value:                     0.000
==============================================================================================
                                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                     -4.1120      0.113    -36.286      0.000      -4.334      -3.890
person_age                     0.0073      0.007      1.110      0.267      -0.006       0.020
person_income               7.437e-05    5.7e-05      1.304      0.192   -3.74e-05       0.000
person_home_ownership          0.2367      0.016     15.234      0.000       0.206       0.267
person_emp_length             -0.0034      0.005     -0.635      0.526      -0.014       0.007
loan_intent                   -0.1414      0.011    -12.400      0.000      -0.164      -0.119
loan_grade                     0.8635      0.037     23.158      0.000       0.790       0.937
loan_amnt                     -0.0041      0.000    -11.986      0.000      -0.005      -0.003
loan_int_rate                  0.0007      0.001      1.152      0.249      -0.000       0.002
loan_percent_income            0.1301      0.005     24.889      0.000       0.120       0.140
cb_person_default_on_file     -0.1126      0.051     -2.219      0.026      -0.212      -0.013
cb_person_cred_hist_length    -0.0065      0.010     -0.641      0.521      -0.026       0.013
==============================================================================================
"""

In [12]:
model.params

Intercept                    -4.111956
person_age                    0.007298
person_income                 0.000074
person_home_ownership         0.236710
person_emp_length            -0.003441
loan_intent                  -0.141380
loan_grade                    0.863485
loan_amnt                    -0.004072
loan_int_rate                 0.000650
loan_percent_income           0.130148
cb_person_default_on_file    -0.112584
cb_person_cred_hist_length   -0.006467
dtype: float64

In [13]:
model.conf_int()[0]

Intercept                    -4.334062
person_age                   -0.005591
person_income                -0.000037
person_home_ownership         0.206255
person_emp_length            -0.014064
loan_intent                  -0.163726
loan_grade                    0.790403
loan_amnt                    -0.004738
loan_int_rate                -0.000456
loan_percent_income           0.119899
cb_person_default_on_file    -0.212014
cb_person_cred_hist_length   -0.026228
Name: 0, dtype: float64

In [14]:
model.conf_int()[1]

Intercept                    -3.889849
person_age                    0.020187
person_income                 0.000186
person_home_ownership         0.267164
person_emp_length             0.007183
loan_intent                  -0.119034
loan_grade                    0.936567
loan_amnt                    -0.003406
loan_int_rate                 0.001756
loan_percent_income           0.140397
cb_person_default_on_file    -0.013154
cb_person_cred_hist_length    0.013294
Name: 1, dtype: float64